In [1]:
n_classes_minicifar = 4
R = 5

# Download the entire CIFAR10 dataset

from torchvision.datasets import CIFAR10
import numpy as np 
from torch.utils.data import Subset
import torchvision.transforms as transforms

## Normalization is different when training from scratch and when training using an imagenet pretrained backbone

normalize_scratch = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

normalize_forimagenet = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

# Data augmentation is needed in order to train from scratch
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize_scratch,
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalize_scratch,
])

## No data augmentation when using Transfer Learning
transform_train_imagenet = transforms.Compose([
    transforms.ToTensor(),
    normalize_forimagenet,
])

transform_test_imagenet = transforms.Compose([
    transforms.ToTensor(),
    normalize_forimagenet,
])


### The data from CIFAR10 will be downloaded in the following dataset
rootdir = './data/cifar10'

c10train = CIFAR10(rootdir,train=True,download=True,transform=transform_train)
c10test = CIFAR10(rootdir,train=False,download=True,transform=transform_test)

c10train_imagenet = CIFAR10(rootdir,train=True,download=True,transform=transform_train_imagenet)
c10test_imagenet = CIFAR10(rootdir,train=False,download=True,transform=transform_test_imagenet)

# Generating Mini-CIFAR
# 
# CIFAR10 is sufficiently large so that training a model up to the state of the art performance will take approximately 3 hours on the 1060 GPU available on your machine. 
# As a result, we will create a "MiniCifar" dataset, based on CIFAR10, with less classes and exemples. 

def generate_subset(dataset,n_classes,reducefactor,n_ex_class_init):

    nb_examples_per_class = int(np.floor(n_ex_class_init / reducefactor))
    # Generate the indices. They are the same for each class, could easily be modified to have different ones. But be careful to keep the random seed! 

    indices_split = np.random.RandomState(seed=42).choice(n_ex_class_init,nb_examples_per_class,replace=False)


    all_indices = []
    for curclas in range(n_classes):
        curtargets = np.where(np.array(dataset.targets) == curclas)
        indices_curclas = curtargets[0]
        indices_subset = indices_curclas[indices_split]
        #print(len(indices_subset))
        all_indices.append(indices_subset)
    all_indices = np.hstack(all_indices)
    
    return Subset(dataset,indices=all_indices)
    
### These dataloader are ready to be used to train for scratch 
minicifar_train= generate_subset(dataset=c10train,n_classes=n_classes_minicifar,reducefactor=R,n_ex_class_init=5000)
minicifar_val= generate_subset(dataset=c10test,n_classes=n_classes_minicifar,reducefactor=1,n_ex_class_init=1000) 
minicifar_test= generate_subset(dataset=c10test,n_classes=n_classes_minicifar,reducefactor=1,n_ex_class_init=1000) 


### These dataloader are ready to be used to train using Transfer Learning 
### from a backbone pretrained on ImageNet
minicifar_train_im= generate_subset(dataset=c10train_imagenet,n_classes=n_classes_minicifar,reducefactor=R,n_ex_class_init=5000)
minicifar_val_im= generate_subset(dataset=c10test_imagenet,n_classes=n_classes_minicifar,reducefactor=1,n_ex_class_init=1000)
minicifar_test_im= generate_subset(dataset=c10test_imagenet,n_classes=n_classes_minicifar,reducefactor=1,n_ex_class_init=1000)


Extracting ./data/cifar10/cifar-10-python.tar.gz to ./data/cifar10
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [0]:
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, size_factor=64):
        super(ResNet, self).__init__()
        self.in_planes = size_factor

        self.conv1 = nn.Conv2d(3, size_factor, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(size_factor)
        self.layer1 = self._make_layer(block, size_factor, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 2*size_factor, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 4*size_factor, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 8*size_factor, num_blocks[3], stride=2)
        self.linear = nn.Linear(8*size_factor*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])

def ResNetCustom(size_factor, num_classes):
    return ResNet(BasicBlock, [2,2,2,2], size_factor=size_factor, num_classes=num_classes)


In [0]:
import math
import torch.nn as nn
import numpy as np
import torch

class PR():
    def __init__(self,model,norm):
        #Model
        self.model = model
        self.weak_params = 0.0
        self.norm = norm
        
    #Computer the weight sum for a kernel/filter
    #filt = conv2.weight[i][j].data)
    def L1(self,kern):
        return np.sum(np.abs(kern.cpu().numpy()))

    def L2(self,kern):
        return np.sqrt(np.sum(np.square(kern.cpu().numpy())))

    def Max(self,kern):
        return np.max(np.abs(kern.cpu().numpy()))

    def Infinity(self,kern):
        conv = kern.cpu().numpy()
        max_value = 0.0
        for c in conv:
            temp = np.sum(np.abs(c))
            if (temp > max_value):
                max_value = temp
        return temp

    #def sum_weights_layer(self,m):
        #input_shape,output_shape = m.in_channels, m.out_channels
        #norm_factor = (1/(input_shape*m.kernel_size[0]*m.kernel_size[1]))
        #sum_layer = [0]*output_shape
        #for j in range(output_shape):
            #for i in range(input_shape):
                #sum_layer[j] += self.sum_weights(m.weight[j][i].data)
            #sum_layer[j] = sum_layer[j]*norm_factor
        #Normalize the sum
        #return sum_layer
    
    def sum_weights_layer(self,m,layer_num):
        input_shape,output_shape = m.in_channels, m.out_channels
        norm_factor = (1/(input_shape*m.kernel_size[0]*m.kernel_size[1]))
        sum_layer = []
        for j in range(output_shape):
            temp,temp_sum = [],0.0
            temp.append(layer_num)
            temp.append(j)
            for i in range(input_shape):
                if self.norm == "L1":
                  temp_sum += self.L1(m.weight[j][i].data)
                elif self.norm == "Max":
                  temp_sum += self.Max(m.weight[j][i].data)
                elif self.norm == "L2":
                  temp_sum += self.L2(m.weight[j][i].data)
                elif self.norm == "Infinity":
                  temp_sum += self.Infinity(m.weight[j][i].data)
            if self.norm == "L1":
                temp_sum = temp_sum*norm_factor
            elif self.norm == "L2":
                temp_sum = temp_sum*norm_factor
            elif self.norm == "Infinity":
                temp_sum = temp_sum*norm_factor*m.kernel_size[0]
            temp.append(temp_sum)
            sum_layer.append(temp)
        return sum_layer

    def extract_min(self,li,ratio):
        sor = sorted(li, key=lambda x: x[2])
        res = sor[: math.floor(len(li)*ratio)]
        return res
    
    #def extract_min(self,sum_layer, ratio):
        #sum_layer_min = sum_layer.copy()
        #number_to_pop = math.floor(len(sum_layer)*ratio)
        #result = []
        #for i in range(number_to_pop):
            #sum_layer_min.pop(np.where(sum_layer_min == np.amax(sum_layer_min))[0][0])   
        #for i in sum_layer_min:
            #result.append(np.where(np.array(sum_layer) == i)[0][0])                    
        #return result   
    
    def zeros_kernel(self,m,weak_filters,index):
        weak_filters = [f for f in weak_filters if f[0]==index]
        for f in weak_filters:
            for i in range(m.in_channels):
                a = torch.from_numpy(np.zeros(m.kernel_size))
                with torch.no_grad():
                    m.weight[f[1]][i].copy_(a)
                    self.weak_params += m.kernel_size[0]*m.kernel_size[1]

    #Prunning on each layer
    def prunning_per_layer(self, ratio):
        #Go through model.modules
        index = -1
        for m in self.model.modules() :
            if isinstance(m,nn.Conv2d):
                index += 1
                sum_layer = self.sum_weights_layer(m,index)
                weak_filters = self.extract_min(sum_layer, ratio)
                self.zeros_kernel(m,weak_filters,index)

    #Prunning on all layers               
    def prunning_net(self,ratio):
        index = -1
        filters = []
        for m in self.model.modules():
            if isinstance(m,nn.Conv2d):
                index += 1
                filters += self.sum_weights_layer(m,index)
        weak_filters = self.extract_min(filters,ratio)
        index = -1
        for m in self.model.modules():
            temp = []
            if isinstance(m,nn.Conv2d):
                index += 1
                self.zeros_kernel(m,weak_filters,index)
         
            
        
        

In [100]:
a = [[1,1],[1,2],[2,1]]
b = [x for x in a if x[0]==1]
print(b)

[[1, 1], [1, 2]]


In [196]:
import torchvision.models

model = torchvision.models.resnet18(True)

import torch
import torch.nn as nn
import torch.nn.functional as F

conv1 = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=(3,3))
p = count_weights(model)
pr = PR(model)

li = pr.prunning_per_layer(0.5)
print(li)
#print(pr.weak_params)
#print(pr.prunning_net(0.5))
#print(pr.weak_params)
#print("Number of paramaters erased : {}".format((pr.weak_params/p)*100))


TypeError: ignored

In [0]:
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.optim as optim
import json

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def main(epoch, iteration, ratios,method, norm):
    
    batch_size = 32
    size_factor = 16
    
    ### These dataloader are ready to be used to train for scratch 
    minicifar_train = generate_subset(dataset=c10train,n_classes=n_classes_minicifar,reducefactor=5,n_ex_class_init=5000)
    trainloader = DataLoader(minicifar_train, batch_size=batch_size, shuffle=True, num_workers=2)
    minicifar_test = generate_subset(dataset=c10test,n_classes=n_classes_minicifar,reducefactor=1,n_ex_class_init=1000)
    testloader = DataLoader(minicifar_test,batch_size=batch_size, num_workers=2)
    minicifar_val = generate_subset(dataset=c10test,n_classes=n_classes_minicifar,reducefactor=5,n_ex_class_init=1000) 
    valloader = DataLoader(minicifar_val, batch_size=batch_size, num_workers=2)
    
    accu_final = []

    for ratio in ratios:
        ### Model is loaded pre-trained
        net = ResNetCustom(size_factor=size_factor, num_classes=4)
        net.load_state_dict(torch.load('./saved_nn/bs32ep300sf16.pth'))
        weights = count_parameters(net)
        net.cuda()

        prunning_net = PR(net,norm)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
    

        correct_test = 0.0
        running_loss_test = 0.0
        total_test = 0.0
        
        for it in range(iteration):
            
            if method == "prunning_layer":
                prunning_net.prunning_per_layer(ratio)
            elif method == "prunning_net":
                prunning_net.prunning_net(ratio)

            ###### FINE-TUNING ######
            for e in range(epoch):

                correct_val = 0.0
                running_loss_val = 0.0
                total_val = 0.0

                net.train()
                for _, (data, labels) in enumerate(trainloader):
                    #setting to cuda
                    data = data.cuda()
                    labels = labels.cuda()

                    # zero the parameter gradient
                    optimizer.zero_grad()

                    # forward + backward + optimize
                    outputs = net(data)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                net.eval()
                for _, (data, labels) in enumerate(valloader):
                    #setting to cuda
                    data = data.cuda()
                    labels = labels.cuda()

                    # compute
                    outputs = net(data)
                    loss = criterion(outputs, labels)

                    # compute statistics
                    total_val += labels.size(0)
                    running_loss_val += loss.item()
                    predicted = outputs.max(1)[1]
                    correct_val += predicted.eq(labels).sum().item()
                    
        ###### RUNNING FINAL TEST ######
        net.eval()
        for _, (data, labels) in enumerate(testloader):
            data = data.cuda()
            labels = labels.cuda()

            #Zero the parameter gradient
            optimizer.zero_grad()
            outputs = net(data)
            running_loss_test = criterion(outputs,labels)
             # compute statistics
            total_test += labels.size(0)
            running_loss_test += loss.item()
            predicted = outputs.max(1)[1]
            correct_test += predicted.eq(labels).sum().item()
            
            ##data = data.cpu()
            ##labels = labels.cpu()
            ##imshow(torchvision.utils.make_grid(data))
            ##print('GroundTruth: ', ' '.join('%5s' % labels[j] for j in range(10)))
            ##print('Predicted: ', ' '.join('%5s' % predicted[j].data for j in range(10)))
            ##print('\n')

        
        print("Method : {} | Norm : {} | Conv Filters Removed : {} % | [Test] Accuracy  : {} % | Weights Removed : {} %".format(method,norm,ratio*100,100*correct_test/total_test, ((prunning_net.weak_params)/weights)*100))
        
        accu_final.append(100*correct_test/total_test)
        

    #### SAVING DATAS ####
    personnal_state_dict = {}
    personnal_state_dict.update({"epoch":epoch, "iteration":iteration, "ratio":ratios, "accu": accu_final, "method":method, "norm":norm})
    
    with open('./save/' + 'ep' + str(epoch) + 'it' + str(iteration) + 'me' + str(method) + 'no' + str(norm) + '.json', 'w') as file:
        file.write(json.dumps(personnal_state_dict))
        
    torch.save(net.state_dict(), './save/' + 'ep' + str(epoch) + 'it' + str(iteration) + 'me' + str(method) + 'no' + str(norm) + '.pth')

In [0]:
import matplotlib.pyplot as plt
import json

def plot(epoch, iteration, ratio):
    
    with open('save/ep' + str(epoch) + 'it' + str(iteration) + 'me' + str(method) + 'no' + str(norm) + '.json', 'r') as file:
        text = file.read()
        jf = json.loads(text)
        
        accu = jf["accu"]
        ratios = jf["ratio"]        
        
    fig, ax = plt.subplots()
    ax.plot(ratio, accu, '-b', label = "Accu function of ratio")

    plt.title("Accuracy")
    plt.text(0.5, 0.4, 'ep' + str(epoch) + 'it' + str(iteration), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=15)
    plt.xlabel("Ratio")
    plt.ylabel("Accuracy (%)")
    ax.legend()
    plt.savefig('accu_' + 'ep' + str(epoch) + 'it' + str(iteration) + 'me' + str(method) + 'no' + str(norm) + '.pdf')

    plt.show()



In [0]:
def plot_multiple(epoch,iteration,ratio,method,norm):
    accu_list = []

    for n in norm:
      with open('save/ep' + str(epoch) + 'it' + str(iteration) + 'me' + str(method) + 'no' + n + '.json', 'r') as file:
          text = file.read()
          jf = json.loads(text)
          
          accu = jf["accu"]
          ratios = jf["ratio"] 
          accu_list.append(accu)

    fig, ax = plt.subplots()
    index = -1
    for a in accu_list:
      index += 1
      ax.plot(ratio, a, label = "Accu | Norm : {} | Method {}".format(norm[index],method))   

    plt.title("Accuracy for different norms")
    #plt.text(0.5, 0.4, 'ep' + str(epoch) + 'it' + str(iteration), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=15)
    plt.xlabel("Ratio")
    plt.ylabel("Accuracy (%)")
    ax.legend()
    plt.savefig('Methods' + ' accu_' + 'ep' + str(epoch) + 'it' + str(iteration) + 'me' + str(method) + 'no' + str(norm) + '.pdf')

    plt.show()   


In [0]:
#ratio = [1,0.95, 0.9, 0.85, 0.8,0.75, 0.7,0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.0]
ratio = [1,0.8,0.6,0.4,0.2,0.0]
norm = ["Max","L1", "Infinity"]
epoch = 10
iteration = 1

method = "prunning_net"
main(epoch,iteration,ratio, method,"Infinity")
main(epoch,iteration,ratio, method,"L1")
main(epoch,iteration,ratio, method,"L2")
plot_multiple(epoch,iteration,ratio,method,norm)

method = "prunning_per_layer"
main(epoch,iteration,ratio, method,"Infinity")
main(epoch,iteration,ratio, method,"L1")
main(epoch,iteration,ratio, method,"L2")
plot_multiple(epoch,iteration,ratio,method,norm)

Method : prunning_net | Norm : Infinity | Conv Filters Removed : 100 % | [Test] Accuracy  : 25.0 % | Weights Removed : 99.58383997533866 %
Method : prunning_net | Norm : Infinity | Conv Filters Removed : 80.0 % | [Test] Accuracy  : 69.3 % | Weights Removed : 97.75935789191257 %
Method : prunning_net | Norm : Infinity | Conv Filters Removed : 60.0 % | [Test] Accuracy  : 66.825 % | Weights Removed : 90.05497422547995 %
Method : prunning_net | Norm : Infinity | Conv Filters Removed : 40.0 % | [Test] Accuracy  : 78.65 % | Weights Removed : 71.0246442088678 %
Method : prunning_net | Norm : Infinity | Conv Filters Removed : 20.0 % | [Test] Accuracy  : 83.125 % | Weights Removed : 39.458135671593226 %
Method : prunning_net | Norm : Infinity | Conv Filters Removed : 0.0 % | [Test] Accuracy  : 80.45 % | Weights Removed : 0.0 %
Method : prunning_net | Norm : L1 | Conv Filters Removed : 100 % | [Test] Accuracy  : 25.0 % | Weights Removed : 99.58383997533866 %
Method : prunning_net | Norm : L1 | C

In [0]:
#IDEAS : 
# Pruning sur les couches du début ? sur les couches de fin ?
# Pruning sur l'ensemble des filtres du Net DONE
# Jouer sur la norme
## imgshow après la première couche